# The web search tool

> Important Note: Your organization must enable the Web Search tool in the settings console before using it. You can find this setting here: https://console.anthropic.com/settings/privacy

Claude includes a built-in web search tool that lets it search the internet for current or specialized information to answer user questions. Unlike other tools where you need to provide the implementation, Claude handles the entire search process automatically - you just need to provide a simple schema to enable it.

In [11]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [12]:
# Helper functions
from anthropic.types import Message

def add_messages(messages, message, role="user"):
    user_message = {
        "role": role,
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)

def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message

def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [13]:
web_search_schema = {
    "type": "web_search_20250305",
    "name": "web_search",
    "max_uses": 5,
    "allowed_domains": ["nih.gov"],
}

In [14]:
messages = []
add_messages(
    messages, 
    """ 
    What's the best exercise for gaining leg muscle?
    """, 
    role="user")
response = chat(messages, tools=[web_search_schema])
response

Message(id='msg_01NMnoGUPa2sQKpR7hR8eyiG', content=[TextBlock(citations=None, text="The best exercises for gaining leg muscle are compound movements that work multiple muscle groups simultaneously. Here are the top choices:\n\n**1. Barbell Back Squat**\nOften considered the king of leg exercises, squats target the quadriceps, hamstrings, glutes, and calves. They allow you to lift heavy weights, which is crucial for muscle growth.\n\n**2. Romanian Deadlifts (RDLs)**\nExcellent for targeting the hamstrings and glutes while also working the lower back. They're particularly effective for posterior chain development.\n\n**3. Bulgarian Split Squats**\nA unilateral exercise that helps address muscle imbalances while heavily targeting the quads and glutes. They're also easier on the lower back than traditional squats.\n\n**4. Leg Press**\nAllows you to safely load heavy weight and isolate the legs without taxing the lower back as much as squats. Great for overall leg mass.\n\n**5. Lunges (Walk